In [254]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 3 – Merging and Pivoting

## DSC 80, Fall 2025

### Due Date: Monday, October 20th at 11:59PM

## Instructions

Welcome to the third DSC 80 lab this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2025-wi).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [255]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [256]:
from lab import *

In [257]:
import os
import io
from pathlib import Path
import pandas as pd
import numpy as np

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_survey` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

## Part 1: Combining Data

### Question 1 – Making Connections 🤝

A group of students decided to send out a survey to their connections on LinkedIn. Each student asks 1000 of their connections for their first and last name, the company they currently work at, their job title, their email, and the university they attended.

**Your job is to combine all the data contained in the files `survey*.csv` (stored within the `data/responses` folder) into a single DataFrame. The number of files and the number of rows in each file may vary, so don't hardcode your answers!** To do so, implement the following two functions.

#### `read_linkedin_survey`

Complete the implementation of the function `read_linkedin_survey`, which takes in a Path for the directory where `survey*.csv` files are located and outputs a single DataFrame with six columns titled `'first name'`, `'last name'`, `'current company'`, `'job title'`, `'email'`, and `'university'` (in that order) containing the all of survey information from all the `.csv` files. Make sure to reset the index of the combined DataFrame before returning it so that the index is unique. When the function takes in an invalide directory it should raise a `FileNotFoundError`.

***Hints***:

- Take a look at a few of the files in the `responses` folder. You may have to do some data cleaning to combine the DataFrames!

- You can list the files in a directory using `.iterdir()`.

***Note***: Remember to use `Path()` to build file paths. Calling `.iterdir()` on a `Path()` object will automatically throw a `FileNotFoundError`.

<br>

#### `com_stats`

Complete the implementation of the function `com_stats`, which takes in a DataFrame returned by `read_linkedin_survey` and returns a list containing, in the following order: 
- The proportion of people who went to a university with the string `'Ohio'` in its name that have the string `'Programmer'` somewhere in their job title.
- The number of job titles that **end** with the exact string `'Engineer'`. Note that we're asking for the number of job titles, **not** the number of people!
- The job title that has the longest name (there are no ties).
- The number of people who have the word `'manager'` in their job title, uppercase or lowercase (`'Manager'`, `'manager'`, and `'mANAgeR'` should all count).

In [258]:
directory = Path('data') / 'responses'
directory = Path(directory)

# Define a function to normalize the columns of a dataframe
def normalize_cols(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('_', ' ')
    return df

# Define a list for which we add dataframes to
dfs = []

# For each path in the directory:
#   1. Read the CSV as a dataframe
#   2. Normalize the columns of the dataframe
#   3. Append the dataframe to the list

for path in list(directory.iterdir()):
    df = pd.read_csv(str(path))
    df = normalize_cols(df)
    dfs.append(df)

# Concatenate the dataframes and reset the index
combined = pd.concat(dfs, ignore_index=True)
combined = combined.reset_index(drop=True)
combined

,first name,last name,job title,email,current company,university
0,Marsh,Kyllford,Financial Analyst,mkyllford0@naver.com,"Abernathy, Brown and Stiedemann",Ganja State University
1,Igor,Greatex,Data Coordiator,igreatex1@phpbb.com,"Stiedemann, Eichmann and Will",Meikai University
2,Garrick,Truscott,Tax Accountant,gtruscott2@weather.com,Collins-Hintz,Saint Paul University
3,Eleanore,Sansam,Senior Quality Engineer,esansam3@pbs.org,Trantow and Sons,Universidad del Rosario
4,Laurent,Bagley,Financial Advisor,lbagley4@statcounter.com,Weissnat and Sons,St. Petersburg StateMechnikov Medical Academy
...,...,...,...,...,...,...
4995,Hurleigh,Sphinxe,Marketing Manager,hsphinxern@eventbrite.com,Kozey-Hamill,Southampton Solent University
4996,Michele,Finch,Account Coordinator,mfinchro@jigsy.com,Ernser and Sons,Glion Institute of Higher Education
4997,Harlie,Heister,Technical Writer,hheisterrp@last.fm,Schuppe-McClure,Tianjin University of Finance & Economics
4998,Phillipe,Dibble,Analog Circuit Design manager,pdibblerq@jugem.jp,"Barrows, Batz and Dickens",University of South Bohemia


In [259]:
# Proportion of people with 'Programmer' in their job title who went to a university with 'Ohio" in it
combined[combined['job title'].str.contains('Programmer') & combined['university'].str.contains('Ohio')].shape[0] / combined.shape[0]
# Number of job titles that end in 'Engineer' 
combined[combined['job title'].str.split().str[-1] == 'Engineer']['job title'].nunique()
# Job title with the longest name
max(combined['job title'].fillna('').unique(), key=len)
# Number of people with manager in their job title
combined['job title'].str.lower().str.contains('manager').sum()

369

In [260]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'responses'
q1_out = read_linkedin_survey(dirname)
stats_out = com_stats(q1_out)

In [261]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2 – Survey Says... 👨‍👩‍👧‍👦

Professor Aritra often sends out extra credit surveys asking students for their favorite animals, movies, and other favorite things. These surveys are stored in the `data/extra-credit-surveys` folder. Each file in that folder corresponds to a different survey question (except for `favorite1.csv`, which contains students' names and IDs).

Here's how extra credit works:
- Each student who has completed at least 50% of the survey questions receives 5 points of extra credit.
- If there is at least one survey question that at least 90% of the class answered (e.g. favorite animal), **everyone** in the class receives 1 point of extra credit. This overall class extra credit only applies twice, so if for example 95% of students answer the favorite color survey question and 91% answer the favorite animal survey question, and and 97% answer the favorite movie question, the entire class still receives 2 extra point as a class, not 3.
- Note that this means that the most extra credit any student can earn is 7 points.

#### `read_student_surveys`

Complete the implementation of the function `read_student_surveys` which takes in a Path describing the directory to a folder containing `favorite*.csv` files and outputs a DataFrame containing all of the survey data combined, indexed by student ID (a value 1-1000).

***Note***: Remember to use `Path()` to build file paths.


<br>

#### `check_credit`

Complete the implementation of the function `check_credit` which takes in a DataFrame returned by `read_student_surveys` and outputs a DataFrame indexed by student ID (a value 1-1000) with two columns:
- `'name'`, containing the name of each student, and
- `'ec'`, containing the number of extra credit points each student earned.

***Note***: For the genres survey, `'(no genres listed)'` does not count as a valid response for receiving extra credit. 

In [262]:
directory = Path('data') / 'extra-credit-surveys'

final_df = None

for path in list(directory.iterdir()):
    df = pd.read_csv(str(path))
    if final_df is None:
        final_df = df
    else:
        final_df = pd.merge(final_df, df, on='id')

final_df = final_df.set_index('id')
final_df

,name,movie,genre,animal,plant,color
id,,,,,,
1,Myrtia,NaN,(no genres listed),Long-crested hawk eagle,NaN,Red
2,Nathanil,NaN,Documentary,Euro wallaby,NaN,Khaki
3,Joni,"Glass-blower's Children, The (Glasblåsarns barn)",NaN,Brown brocket,NaN,Red
4,Prentice,NaN,(no genres listed),"Peccary, white-lipped",NaN,Yellow
5,Claudette,NaN,NaN,"Capuchin, brown",NaN,Fuscia
...,...,...,...,...,...,...
996,Addie,Kung Phooey!,Horror|Mystery|Sci-Fi,"Eland, common",NaN,Purple
997,Valaria,Angel Heart,NaN,Agouti,NaN,Blue
998,Gunilla,NaN,NaN,"Shelduck, european",NaN,NaN


In [307]:
final_df_copy = final_df.copy()
final_df_copy = final_df_copy.replace('(no genres listed)', np.nan)
final_df_copy = final_df_copy.reset_index()
final_df_copy = final_df_copy.set_index(['id', 'name'])
final_df_copy['ec'] = final_df_copy.isna().mean(axis=1) < 0.5
final_df_copy['ec'] = final_df_copy['ec'].map({True: 5, False: 0})


class_ec = min((final_df_copy.isna().mean(axis=0) < 0.1).sum() - 1, 2)
final_df_copy['ec'] += class_ec

final_df_copy = final_df_copy.reset_index()
final_df_copy[['name', 'ec']]

,name,ec
0,Myrtia,1
1,Nathanil,6
2,Joni,6
3,Prentice,1
4,Claudette,1
...,...,...
995,Addie,6
996,Valaria,6
997,Gunilla,1
998,Zitella,1


In [268]:
# Step 1 - for each student, check whether they answered at least half the questions
final_df_copy = final_df.copy()
final_df_copy = final_df_copy.replace('(no genres listed)', np.nan)
final_df_copy['ec'] = final_df.isna().mean(axis=1) < 0.5
final_df_copy['ec'] = final_df_copy['ec'].map({True: 5, False: 0})

# Step 2 - check how many questionos had more than 90% of the class answer
class_ec = min((final_df.isna().mean(axis=0) < 0.1).sum(), 2)

final_df_copy['ec'] += class_ec
final_df_copy[['name', 'ec']]

,name,ec
id,,
1,Myrtia,7
2,Nathanil,7
3,Joni,7
4,Prentice,7
5,Claudette,2
...,...,...
996,Addie,7
997,Valaria,7
998,Gunilla,2


In [308]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'extra-credit-surveys'
q2_out = read_student_surveys(dirname)
check_credit_out = check_credit(q2_out)

In [ ]:
grader.check("q2")

q2 results: All test cases passed!

: 

### Question 3 – Paw Patrol 🐾

You are analyzing data from a veterinarian clinic. The datasets contain several types of information from the clinic, including its customers (pet owners), pets, available procedures, and procedure history. The column names are self-explanatory. These DataFrames are provided to you:
-  `owners` stores the customer information, where every `'OwnerID'` is unique (verify this yourself).
-  `pets` stores the pet information. Each pet belongs to a customer in `owners`.
-  `procedure_detail` contains a catalog of procedures that are offered by the clinic.
-  `procedure_history` has procedure records. Most procedures were given to a pet in `pets`.

Complete the implementation of the following three functions, which each ask you to answer a specific question.

#### `most_popular_procedure`

What is the most popular `'ProcedureType'` amongst all pets in the `pets` DataFrame? Complete the implementation of the function `most_popular_procedure`, which takes in two DataFrames, `pets` and `procedure_history`, and returns the name of the most popular `'ProcedureType'` as a string.

Note that some pets are registered but haven't had any procedures performed. Also, some pets that have had procedures done are not registered in `pets`.

<br>

#### `pet_name_by_owner`

What is the name of each customer's pet(s)? Complete the implementation of the function `pet_name_by_owner`, which takes in two DataFrames, `owners` and `pets`, and returns a Series whose index contains owner first names, and whose values are pet names as **strings**. If an owner has multiple pets, the value corresponding to that owner should instead be a **list of pet names as strings**.

Note that owner first names are not necessarily unique, and so the Series you return will not necessarily have a unique index.

<br>

#### `total_cost_per_city`

Note that the `owners` DataFrame has a `'City'` column, describing the city in which each pet owner and their pets live. How much did each city spend in total on procedures? Complete the implementation of the function `total_cost_per_city`, which takes in four DataFrames, `owners`, `pets`, `procedure_history`, and `procedure_detail`, and returns a Series indexed by `'City'` that describes the total amount that each city has spent on pets' procedures.

***Hint***: At some point, you may have to merge on multiple columns.

***Note***: Some owners may have never visited the veterinarian clinic in their city. This means some cities may have zero operational costs.

In [234]:
# do not edit this cell -- it is needed for the tests
pets_fp = Path('data') / 'pets' / 'Pets.csv'
procedure_history_fp =  Path('data') / 'pets' / 'ProceduresHistory.csv'
owners_fp = Path('data') / 'pets' / 'Owners.csv'
procedure_detail_fp = Path('data') / 'pets' / 'ProceduresDetails.csv'
pets = pd.read_csv(pets_fp)
procedure_history = pd.read_csv(procedure_history_fp)
owners = pd.read_csv(owners_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)

In [235]:
# Determine the most popular procedure among pets in the pet dataset
# Inner join to allow for easy analysis of pets that have had procedures done on them
pet_procedures = pd.merge(pets, procedure_history, how='inner', on='PetID')
pet_procedures['ProcedureType'].value_counts().sort_values(ascending=False).index[0]

'VACCINATIONS'

In [236]:
owners_and_pets = pd.merge(owners, pets, how='inner', on='OwnerID')

def get_str_or_list(ser):
    if len(ser) == 1:
        return ser.iloc[0]
    return list(ser)

owners_and_pets_grouped = pd.DataFrame(owners_and_pets.groupby(['OwnerID', 'Name_x', 'Surname'])['Name_y'].agg(get_str_or_list))
owners_and_pets_grouped = owners_and_pets_grouped.reset_index().drop(columns=['OwnerID', 'Surname'])
owners_and_pets_grouped.columns = ['Owner Name', 'Pet Name(s)']
owners_and_pets_grouped = owners_and_pets_grouped.set_index('Owner Name')
owners_and_pets_grouped

,Pet Name(s)
Owner Name,
Jessica,Biscuit
Rosa,Stowe
Susan,Enyo
Benjamin,"[Danger, Collette]"
Charles,Rumba
...,...
William,Bandit
Carmen,Maripol
Steven,Rumba


In [237]:
procedure_history

,PetID,Date,ProcedureType,ProcedureSubCode
0,A8-1181,2016-01-10,VACCINATIONS,5
1,E7-3766,2016-01-11,VACCINATIONS,5
2,B8-8740,2016-01-11,VACCINATIONS,5
3,D4-9443,2016-01-11,VACCINATIONS,5
4,F6-3398,2016-01-12,HOSPITALIZATION,1
...,...,...,...,...
2279,A4-2573,2016-12-30,VACCINATIONS,5
2280,B3-8317,2016-12-30,VACCINATIONS,5
2281,C5-4291,2016-12-30,ORTHOPEDIC,3
2282,C6-3299,2016-12-31,VACCINATIONS,5


In [238]:
detailed_procedures = pd.merge(procedure_history, procedure_detail, how='inner', on=['ProcedureType', 'ProcedureSubCode'])
pets_detailed_procedures = pd.merge(pets, detailed_procedures, how='left', on='PetID')
owners_pets_detailed_procedures = pd.merge(owners, pets_detailed_procedures, how='inner', on='OwnerID')
owners_pets_detailed_procedures.groupby('City')['Price'].sum().sort_values(ascending=False)

City
Grand Rapids           1240.0
Ann Arbor               450.0
Detroit                 305.0
Southfield               65.0
Marquette                50.0
East Lansing             40.0
Pontiac                  30.0
Lansing                  30.0
Flint                    15.0
Warren                   10.0
Saint Charles            10.0
Roseville                10.0
Michigan Center          10.0
Commerce                 10.0
Center Line              10.0
Plymouth                 10.0
Livonia                  10.0
Kalamazoo                10.0
Farmington Hills         10.0
Wayne                    10.0
Bloomfield Township       0.0
Caro                      0.0
Bridgman                  0.0
Bruce Crossing            0.0
Clam River                0.0
Lincoln Park              0.0
Grand Ledge               0.0
Highland Park             0.0
Fruitport                 0.0
Davison                   0.0
Farmington                0.0
Fremont                   0.0
Dutton                    0.0
Hudso

In [239]:
procedure_detail

,ProcedureType,ProcedureSubCode,Description,Price
0,OFFICE FEES,1,Office Call,32
1,OFFICE FEES,2,Emergency,100
2,OFFICE FEES,3,Reck,24
3,GROOMING,1,Bath,15
4,GROOMING,2,Flea Dip,15
5,GROOMING,3,Flea Spray,10
6,VACCINATIONS,1,Galaxie (DHLPP),15
7,VACCINATIONS,2,Leukemia,20
8,VACCINATIONS,3,Lyme,15
9,VACCINATIONS,4,PCR,15


In [240]:
out_01 = most_popular_procedure(pets, procedure_history)
out_02 = pet_name_by_owner(owners, pets)
out_03 = total_cost_per_city(owners, pets, procedure_history, procedure_detail)

In [241]:
# do not edit this cell -- it is needed for the tests
pets_fp = Path('data') / 'pets' / 'Pets.csv'
procedure_history_fp =  Path('data') / 'pets' / 'ProceduresHistory.csv'
owners_fp = Path('data') / 'pets' / 'Owners.csv'
procedure_detail_fp = Path('data') / 'pets' / 'ProceduresDetails.csv'
pets = pd.read_csv(pets_fp)
procedure_history = pd.read_csv(procedure_history_fp)
owners = pd.read_csv(owners_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)

out_01 = most_popular_procedure(pets, procedure_history)
out_02 = pet_name_by_owner(owners, pets)
out_03 = total_cost_per_city(owners, pets, procedure_history, procedure_detail)

In [242]:
grader.check("q3")

q3 results: All test cases passed!

## Part 2: Pivot Tables

### Question 4 – Summarizing Sales 💰

Recall from [Lecture 3](https://dsc80.com/resources/lectures/lec03/lec03-filled.html#Pivot-tables-using-the-pivot_table-method), a pivot table allows you to aggregate the entries in a DataFrame based on two categorical columns. In this question, you are given a simple dataset, `data/sales.csv`, and are asked to solve a few problems using the `pivot_table` method. 

**We have provided outlines for the DataFrames you need to create, but yours may have a different number of rows and columns and different values.**

***Note***: If it helps, [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) is another great resource that provides an overview of `pivot_table` with many examples from the Titanic dataset.


#### `average_seller`

Complete the implementation of the function `average_seller`, which takes in the `sales` DataFrame and returns a DataFrame that contains the average sales for each seller, indexed by `'Name'` and containing the column `'Average Sales'`. There should not be any `NaN`s.

***Note***: You may be able to implement `average_seller` without using `pivot_table`.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Average Sales</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Complete the implementation of the function `product_name` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each product, indexed by `'Name'`. Do not fill in `NaN`s.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>Product</th>
      <th>boat</th>
      <th>book</th>
      <th>hotel</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Jones</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>NaN</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
  </tbody>
</table>

<br>

#### `count_product`

Complete the implementation of the function `count_product` that takes in the `sales` DataFrame and returns a DataFrame that contains the total number of items sold product-wise and name-wise per date. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Date</th>
      <th>01.01.2012</th>
      <th>02.20.2013</th>
      <th>02.25.2015</th>
    </tr>
    <tr>
      <th>Product</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>boat</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">book</th>
      <th>Jones</th>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Smith</th>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>hotel</th>
      <th>Trump</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `total_by_month`

Complete the implementation of the function `total_by_month` that takes in the `sales` DataFrame and returns a pivot table that contains the total sales name-wise, product-wise per month. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th>Month</th>
      <th>February</th>
      <th>January</th>
      <th>July</th>
      <th>March</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="3" valign="top">Jones</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">Smith</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

In [252]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'sales.csv'
sales = pd.read_csv(fp)
q4_average_seller_out = average_seller(sales)
q4_product_name_out = product_name(sales)
q4_product_count_out = count_product(sales)
q4_total_by_month_out = total_by_month(sales)

In [250]:
average_seller_pivot = sales.pivot_table(values='Total', index='Name', aggfunc='mean')
average_seller_pivot.columns = ['Average Sales']
average_seller_pivot

,Average Sales
Name,
Jones,920.0
Smith,1700.0
Trump,487.5


In [245]:
sales.pivot_table(index='Name', columns='Product', values='Total', aggfunc='sum')

Product,boat,book,hotel,pen,ruler
Name,,,,,
Jones,NaN,30.0,NaN,300.0,3350.0
Smith,NaN,2200.0,NaN,2500.0,2100.0
Trump,700.0,1000.0,100.0,150.0,NaN


In [246]:
sales.pivot_table(index=['Product', 'Name'], columns='Date', values='Total', aggfunc='sum').fillna(0)

Date           01.01.2012  02.20.2013  02.25.2015  03.03.2015  03.30.2017  \
Product Name                                                                
boat    Trump         0.0         0.0         0.0         0.0       700.0   
book    Jones         0.0        30.0         0.0         0.0         0.0   
        Smith       200.0         0.0         0.0         0.0         0.0   
        Trump         0.0         0.0         0.0      1000.0         0.0   
hotel   Trump         0.0         0.0         0.0       100.0         0.0   
pen     Jones         0.0       300.0         0.0         0.0         0.0   
        Smith      2500.0         0.0         0.0         0.0         0.0   
        Trump         0.0         0.0         0.0       150.0         0.0   
ruler   Jones         0.0      3000.0       350.0         0.0         0.0   
        Smith         0.0         0.0         0.0         0.0         0.0   

Date           05.10.2013  07.05.2014  
Product Name                           
boat    Trump         0.0         0.0  
book    Jones         0.0         0.0  
        Smith      2000.0         0.0  
        Trump         0.0         0.0  
hotel   Trump         0.0         0.0  
pen     Jones         0.0         0.0  
        Smith         0.0         0.0  
        Trump         0.0         0.0  
ruler   Jones         0.0         0.0  
        Smith         0.0      2100.0

In [247]:
sales_copy = sales.copy()
sales_copy['Month'] = sales_copy['Date'].str.split('.').str[0]
month_map = {
    '01': 'January', '02': 'February', '03': 'March',
    '04': 'April', '05': 'May', '06': 'June',
    '07': 'July', '08': 'August', '09': 'September',
    '10': 'October', '11': 'November', '12': 'December'
}
sales_copy['Month'] = sales_copy['Month'].map(month_map)
sales_copy.pivot_table(index=['Product', 'Name'], columns='Month', values='Total', aggfunc='sum').fillna(0)

Month          February  January    July   March     May
Product Name                                            
boat    Trump       0.0      0.0     0.0   700.0     0.0
book    Jones      30.0      0.0     0.0     0.0     0.0
        Smith       0.0    200.0     0.0     0.0  2000.0
        Trump       0.0      0.0     0.0  1000.0     0.0
hotel   Trump       0.0      0.0     0.0   100.0     0.0
pen     Jones     300.0      0.0     0.0     0.0     0.0
        Smith       0.0   2500.0     0.0     0.0     0.0
        Trump       0.0      0.0     0.0   150.0     0.0
ruler   Jones    3350.0      0.0     0.0     0.0     0.0
        Smith       0.0      0.0  2100.0     0.0     0.0

In [253]:
grader.check("q4")

q4 results: All test cases passed!

## Congratulations! You're done Lab 3! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the lab, you should open the command line and run, in the directory for this lab:

```
python lab-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `lab.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `lab.py`!**

You can also use `lab-validation.py` to test individual questions. For instance,

```
python lab-validation.py q1 q2 q4
```

will run the `grader.check` cells for Questions 1, 2, and 4 – again, only using the code in `lab.py`. [This video](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=5acc92e6-0dfe-4555-9b6a-8115b6a52f99) how to use the script as well.

Once `python lab-validation.py` shows that you're passing all test cases, you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_survey` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()